# Analyze results for 3D images
Jan 21, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_3d_image_analysis import *

# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
# import post_analysis_pandas as post

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [27]:
img_size=128
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
       '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/'}


### Read validation data and compute metrics

In [28]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy',
                 '128':raw_data_dir+'3d_data/dataset3_smoothing_const_params_128cube/val.npy'}

In [29]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
s_val=f_transform(s_val)
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

# del(s_val)

(500, 128, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/modules_3d_image_analysis.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


### Read run data

In [30]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [31]:
u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)

In [32]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202105*')]
q=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(q)

interactive(children=(Dropdown(description='x', options=('20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_b…

In [11]:
result=q.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210515_224342_128cube_bs4_lr0.0002_nodes2_cori_bestrun


## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
34991,34991.0,174.0,0.524365,0.674484,1.198849,1.708335,14.075881,12.367546,-1.077996,12.270939,-1.002788,NaN,NaN,1.359293,-0.065896,-1.505269,0.708192
34992,34992.0,174.0,0.603182,0.532444,1.135626,0.838857,12.812587,11.973729,-0.993209,11.960321,-0.949831,NaN,NaN,0.114266,-1.128456,-0.254315,0.709322
34993,34993.0,174.0,0.571922,0.681666,1.253588,1.640161,13.894066,12.253905,-1.141080,12.353355,-1.227987,NaN,NaN,1.326950,0.034017,-1.420353,0.711441
34994,34994.0,174.0,0.626902,0.652113,1.279015,0.967395,12.960476,11.993080,-0.956611,11.824598,-1.013767,NaN,NaN,0.231552,-1.261447,-0.480989,0.711078
34995,34995.0,174.0,0.622136,0.626732,1.248868,2.069319,inf,inf,-1.082462,12.127693,-0.971334,NaN,NaN,1.177781,-0.445078,-1.927115,0.707153
34996,34996.0,174.0,0.668509,0.514418,1.182927,0.928720,12.395007,11.466287,-0.946707,12.139159,-1.132594,NaN,NaN,0.077288,-1.361392,-0.392726,0.711509
34997,34997.0,174.0,0.636947,0.627815,1.264762,2.028023,14.036093,12.008070,-0.954272,12.315535,-1.454507,NaN,NaN,1.168278,-0.210208,-1.880281,0.708562
34998,34998.0,174.0,0.613877,0.595733,1.209610,1.011605,12.761950,11.750344,-0.919642,12.010004,-1.197280,NaN,NaN,0.188298,-1.532598,-0.553603,0.708497
34999,34999.0,174.0,0.539795,0.607699,1.147494,2.125350,14.390821,12.265471,-0.732779,12.125668,-0.625554,NaN,NaN,1.245184,-0.635380,-1.994728,0.708203
35000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.163752,-0.667138,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [15]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.6354150974750519


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
24465,24465.0,122.0,0.559914,0.665092,1.225006,1.955065,14.154474,12.199409,-0.995098,11.414593,-2.385275,NaN,NaN,1.304440,-0.017618,-1.792363,0.708562
34822,34822.0,174.0,0.618404,0.585304,1.203708,0.848479,12.992687,12.144208,-1.909964,12.059822,-2.154338,NaN,NaN,0.300781,-1.146129,-0.230887,0.708166
34842,34842.0,174.0,0.630909,0.488403,1.119312,0.926395,13.116182,12.189788,-1.638387,12.231131,-2.126356,NaN,NaN,0.658786,-1.220330,-0.304129,0.708780
34592,34592.0,172.0,0.571527,0.628354,1.199881,1.810578,inf,inf,-1.483608,12.332520,-2.079963,NaN,NaN,1.439438,-1.177440,-1.607465,0.710763
34794,34794.0,173.0,0.621640,0.509527,1.131168,0.978884,13.233103,12.254219,-1.671563,12.354265,-2.060592,NaN,NaN,0.374828,-1.584391,-0.438563,0.708741
34823,34823.0,174.0,0.543397,0.705886,1.249282,1.958575,14.269523,12.310947,-1.657106,11.799654,-2.049083,NaN,NaN,1.366135,-0.150719,-1.785671,0.709302
34811,34811.0,174.0,0.644275,0.674090,1.318366,2.077921,13.676616,11.598695,-1.535474,11.858579,-2.042706,NaN,NaN,1.211559,-0.382256,-1.937009,0.708194
24478,24478.0,122.0,0.655087,0.430148,1.085236,0.873549,12.938417,12.064868,-1.131173,11.992181,-2.039683,NaN,NaN,0.048419,-1.288278,-0.256388,0.707409
31399,31399.0,156.0,0.529722,0.860457,1.390179,2.039229,14.162533,12.123303,-1.325517,12.092537,-1.998089,NaN,NaN,1.386813,-0.163858,-1.898130,0.708982
33696,33696.0,168.0,0.627389,0.523985,1.151374,2.204315,inf,inf,-1.103174,inf,-1.995457,NaN,NaN,1.279423,-0.241379,-2.063117,0.707569


In [16]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [17]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [18]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [19]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [20]:
df_sliced=w.result
# df_sliced

In [21]:
best_step=[]
df_test=df.copy()
df_test=df[df.epoch>30]
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1b',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.4,sort_col='chi_spec1',head=4,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 4.199715214926997, 'chi_spec1': 5.080956792073642}
[0, 175]
{'chi_1': 4.199715214926997, 'chi_spec1': 5.080956792073642}
[0, 175]
{'chi_1': 1.6123545919310693, 'chi_spec1': 1.916903633455336}
[array([7270, 7480, 7370, 6730]), array([6270, 7270]), array([33330, 33280, 33840, 34200])]
[ 6270  6730  7270  7370  7480 33280 33330 33840 34200]


In [22]:
# best_step=[19390]
# best_step=np.arange(32800,32900,10).astype(int)

In [23]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(9, 22)
[(31, 6270), (33, 6730), (36, 7270), (36, 7370), (37, 7480), (166, 33280), (166, 33330), (169, 33840), (170, 34200)]


In [24]:
df_best[col_list]


,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
626,31-6270,0.103761,4.025057,58.592305,0.100296,0.003457,0.000007,0.000586,5.550256,2474.602515,31,6270
672,33-6730,0.099740,3.094251,59.236286,0.055525,0.044208,0.000007,0.000316,10.516804,1814.609887,33,6730
726,36-7270,0.088081,4.139697,60.627927,0.084058,0.004016,0.000007,0.000472,6.263832,6773.720009,36,7270
736,36-7370,0.099121,3.625845,59.280519,0.092617,0.006496,0.000007,0.000513,5.089128,5228.056625,36,7370
747,37-7480,0.094133,4.188665,59.046263,0.089902,0.004224,0.000007,0.000569,7.915122,904.195568,37,7480
3327,166-33280,0.749431,0.492141,61.586716,0.021758,0.727666,0.000007,0.000154,18.616018,274.208147,166,33280
3332,166-33330,0.940773,0.452042,56.608315,0.021857,0.691664,0.227253,0.000151,15.879161,275.660297,166,33330
3383,169-33840,1.449616,0.514262,58.940225,0.012331,1.437273,0.000012,0.000081,24.337763,435.545688,169,33840
3419,170-34200,0.828722,0.514825,54.016696,0.014186,0.814528,0.000009,0.000075,22.066690,474.253494,170,34200


### Plots 2: stored data

In [25]:

def f_plot_hist_spec(df,dict_bkg,plot_type,img_size):

    
    assert plot_type in ['hist','spec','grid','spec_relative','hist_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative','hist_relative']:     fig=plt.figure(figsize=(6,6))


    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            
            
        if plot_type=='hist_relative':

            x2=row.hist_bin_centers
            y2=row.hist_val
            yerr2=row.hist_err
            x2=f_invtransform(x2)
            
#             plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['hist_val'],dict_bkg['hist_err']

            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            plt.xlim(0,img_size/2)
            plt.ylim(0,5)
            plt.legend()

        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            print(images.shape)
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.xlim(0,img_size/2)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)
        plt.legend()
        
    if plot_type=='hist_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [26]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type,img_size)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative','hist_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('31-6270', '33-6730', '36-7270', '36-…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [ ]:
print(s_val.shape)


In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210123_050147_3d_l0.5_80k/images/gen_img_epoch-28_step-71620.npy'
a1=np.load(fname)

In [ ]:
s_val.shape,a1.shape

In [ ]:
f_plot_grid(a1[6:14,:,:,0],cols=4,fig_size=(4,2))

In [ ]:
f_plot_grid(s_val[100:108,:,:,0],cols=4,fig_size=(4,2))

In [ ]:
df_best[col_list]

In [ ]:
dims = [2,3,4,7]
np_x = np.random.uniform(size=dims).astype(np.float32)

In [ ]:
90000/(32*32)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy'
np.load(fname,mmap_mode='r').shape
